In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="mps"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("downloaded")

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

downloaded


RuntimeError: isin_Tensor_Tensor_out only works on floating types on MPS for pre MacOS_14_0. Received dtype: Long

In [8]:
prompt = "Привет, реши 2+2"
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

'Привет! Решение: 2 + 2 = 4'

In [6]:
from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM
model = AutoPeftModelForCausalLM.from_pretrained(
    "/Users/teasgen/workspace/diploma/llm_backend/lora_model",
    load_in_4bit = False,
)
tokenizer = AutoTokenizer.from_pretrained("lora_model")
model.to("mps")
model.eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.38s/it]


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151666, 2048, padding_idx=151665)
        (layers): ModuleList(
          (0-35): 36 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(

In [7]:
prompt = """
> Question: Что такое решающее дерево?
> Reply: This is a family of models, which is represented as a tree, at the vertices of which the rules for dividing the sample are located."""
messages = [
    {"role": "system", "content": "You're conducting an online interview. Here is an excerpt from it. Please comment on the user's response. Give him a score from 0 to 10 (where 0 is an irrelevant answer, 10 is an excellent answer, and there is nothing else to add). If the answer is incomplete, come up with a clarifying question and write it down."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

'Score: 5\n\nExplanation: The user provided a basic definition of what a decision tree is, but they did not elaborate on its key features or how it works in machine learning. A more comprehensive answer would include details such as how decision trees work, their advantages and disadvantages, and some common applications. \n\nClarifying question: Could you please provide more information about how decision trees are built and trained?'